# LDA主题发现

    0、下面主要是通过LDA对文档进行主题发现，操作过程如下。
    
    1、加载语料，语料未分词；加载停用词。
    2、为了能够简化后续计算，希望对每个语料进行关键词提取，并利用关键词进行后续操作。关键词提取步骤：
        2.1 关键词采用的TF-IDF对词进行排序的思路。
        2.2 因此生成每个词的在整个语料中的出现次数的dict。
        2.3 逐个语料计算其中每个词汇的TF-IDF值，并根据所需数目提取前k个词作为该语料的关键词。
    3、得到关键词后，重新生成词袋模型（LDA采用的是词袋模型）
    4、定义需要区分的主题数，跑就行了。
    5、对于每个语料，将其转换为词袋模型，即可进行主题的划分。

In [8]:
import pandas as pd
import numpy as np
import jieba as jb
import jieba.posseg as pseg
import jieba.analyse
from gensim import corpora, models
import matplotlib.pyplot as plt
import math


In [70]:
'''
加载原始语料数据
'''
raw_data=pd.read_csv('left_11335.csv')


In [71]:
'''
加载停用词
'''
stop_list=[]
fff=open('stop_words.txt','r',encoding='utf-8')
for li in fff:
    stop_list.append(li[:-1])
fff.close()


# 提取关键词


    0、需要更新自己的tf-idf库
    1、将语料进行分词，去除停用词，去除指定词性之外的词，得到每个语料的分词结果。
    2、计算每个词在整个语料中出现的次数，作为自定义的IDF表。
    3、将自定义的IDF表导入jieba，利用结巴的extract（背后是TF-IDF），来得到每个语料关键词。

In [72]:
'''
得到自定义的idf表格
'''
all_dict = {}
total=0
final_dict={}
new_line=[]
weibos=raw_data['title'].tolist()
# 定义需要的词性
target_attr=['n','ns','nr','nt','nz','v','a']
for i,line in enumerate(weibos):
    temp_dict = {}
    # 统计有多少文档
    total += 1
    # 分词
    cut_line = pseg.cut(line)
    # 该文档的词语去重
    for word,flag in cut_line:
        # 过滤停用词，以及过滤非指定词性
        if word not in stop_list and flag in target_attr:
            temp_dict[word] = 1
    new_line.append(''.join(temp_dict))
    # 更新这个词在每个文档中出现过多少次
    for key in temp_dict.keys():
        num = all_dict.get(key, 0)
        all_dict[key] = num + 1

# 计算每个词的TF-IDF值。
for key in all_dict:
    w = key
    p = '%.10f' % (math.log10(total/(all_dict[key] + 1)))
    final_dict[w]=p

fff=open('my_idf.txt.big','w',encoding='utf-8')
fff.write( '\n'.join([ w+' '+f for w,f in zip(final_dict.keys(),final_dict.values())]))
fff.close()

In [24]:
final_dict.keys()

dict_keys(['推送', '分', '泰国', '曼谷', '市中心', '发生', '爆炸', '旅游景点', '记者', '现场', '伤亡', '话', '说', '保守', '旧', '名字', '英国人', '英国', '公布', '新生儿', '姓名', '调查报告', '权力', '游戏', '人物', '宝宝', '起名', '妹纸', '做', '素颜', '图', '约会', '软件', '男人', '反差', '数量', '差', '感受', '体验', '伦敦', '电影', '只能', '啃', '爆米花', '号称', '电影院', '伙食', '自制', '美食', '观影', '神秘', '小盒子', '放', '美味', '小吃', '精心', '编', '号码', '让你在', '精彩', '时刻', '享受', '味蕾', '视觉', '双重', '放映', '想去', '介绍', '世界', '年龄', '四胞胎', '母亲', '生', '孩子', '最小', '红', '吸血', '猫', '虎牙', '天生', '主人', '恶煞', '萌', '脸', '对得起', '有没有', '想', '西瓜', '哥哥', '米其林', '餐厅', '请到', '厨', '超', '迎接', '回家', '可爱', '妹', '抖', '节目', '好玩', '抽奖', '环节', '拍', '精美', '手', '办', '等你拿', '主', '爱心', '转发', '微博', '抽', '支付宝', '扮演', '蝙蝠侠', '儿童医院', '探望', '生病', '小朋友', '视频', '响应', '慈善', '团体', '邀请', '带来', '勇气', '高速公路', '故障', '蝙蝠', '车', '撞', '身亡', '英雄', '蜡烛', '穿着', '内衣', '站', '繁华', '广场', '爱惜', '身体', '事情', '鼓励', '读书', '温馨', '公交车', '读', '免票', '流浪狗', '喂成', '大肥', '汪', '华丽', '瘦身', '成功', '美女', '学业成绩', '理想', '担心', '机会', '名校', '学习', '集团', '提供', '包

In [73]:
'''
加载自定义的idf表格，并进行关键词提取
'''
jb.analyse.set_idf_path('my_idf.txt.big')
key_words=[]
for i in range(raw_data.shape[0]):
    weibo=raw_data.loc[i,'title']
    weibo=''.join(weibo.split())
    
    k_w=jieba.analyse.extract_tags(weibo, topK=15, withWeight=False,allowPOS=('n','ns','nr',
                                                                                'nt','nz','ng','i','v','a'))
    k_w=[wd for wd in k_w if str(wd) not in stop_list]
    key_words.append(k_w)
    

In [74]:
raw_data['key_words']=key_words

# 生成LDA模型

    1、根据每个语料的关键词，生成语料的词袋表示
    2、训练lda模型，并展示。
    3、对每个语料进行预测，选择概率最大的主题作为他们的分类

In [101]:
'''
生成语料的关键词表示
生成语料的词袋表示
'''
# 词袋字典
word_dict=corpora.Dictionary(key_words)
print(len(word_dict))
# 语料的词袋表示
corpus_list = [word_dict.doc2bow(text) for text in key_words]
print(len(corpus_list))

21710
11335


In [91]:
'''
训练lda模型
'''
lda = models.ldamodel.LdaModel(corpus=corpus_list,id2word=word_dict,num_topics=10,alpha='auto')

In [93]:
'''
展示头5个模型
'''
lda.print_topics(10)

[(0,
  '0.027*"视频" + 0.017*"拍" + 0.010*"秒" + 0.008*"英国" + 0.008*"胡椒" + 0.008*"孩子" + 0.006*"小哥" + 0.005*"笑" + 0.005*"吃" + 0.005*"妈妈"'),
 (1,
  '0.054*"秒" + 0.048*"视频" + 0.047*"英国" + 0.044*"事儿" + 0.043*"拍" + 0.013*"屎" + 0.013*"铲" + 0.011*"笑" + 0.011*"官" + 0.008*"允悲"'),
 (2,
  '0.017*"波" + 0.014*"网友" + 0.013*"笑" + 0.012*"摊手" + 0.011*"视频" + 0.009*"拍" + 0.007*"推" + 0.007*"开心" + 0.007*"说" + 0.006*"允悲"'),
 (3,
  '0.012*"抱抱" + 0.011*"英国" + 0.009*"妹子" + 0.006*"小哥" + 0.005*"成" + 0.005*"说" + 0.004*"视频" + 0.004*"萌" + 0.004*"世界" + 0.004*"唱"'),
 (4,
  '0.013*"心" + 0.008*"美好" + 0.006*"好看" + 0.006*"世界" + 0.006*"看似" + 0.005*"网友" + 0.005*"故事" + 0.005*"照片" + 0.005*"操作" + 0.004*"袭"'),
 (5,
  '0.012*"英国" + 0.007*"视频" + 0.007*"伦敦" + 0.007*"偷笑" + 0.005*"妹子" + 0.005*"精致" + 0.004*"世界" + 0.004*"现场" + 0.004*"强奸" + 0.004*"吃"'),
 (6,
  '0.018*"跪" + 0.010*"英国" + 0.010*"网友" + 0.008*"小哥" + 0.007*"想" + 0.006*"笑" + 0.006*"说" + 0.006*"画风" + 0.006*"做" + 0.005*"人生"'),
 (7,
  '0.014*"可爱" + 0.009*"开心" + 0.009*"妹子" + 0.008*"

In [94]:
'''
展示第i个主题
'''
for i in range(10):
    print(i,lda.print_topic(i,5))
    print()

0 0.027*"视频" + 0.017*"拍" + 0.010*"秒" + 0.008*"英国" + 0.008*"胡椒"

1 0.054*"秒" + 0.048*"视频" + 0.047*"英国" + 0.044*"事儿" + 0.043*"拍"

2 0.017*"波" + 0.014*"网友" + 0.013*"笑" + 0.012*"摊手" + 0.011*"视频"

3 0.012*"抱抱" + 0.011*"英国" + 0.009*"妹子" + 0.006*"小哥" + 0.005*"成"

4 0.013*"心" + 0.008*"美好" + 0.006*"好看" + 0.006*"世界" + 0.006*"看似"

5 0.012*"英国" + 0.007*"视频" + 0.007*"伦敦" + 0.007*"偷笑" + 0.005*"妹子"

6 0.018*"跪" + 0.010*"英国" + 0.010*"网友" + 0.008*"小哥" + 0.007*"想"

7 0.014*"可爱" + 0.009*"开心" + 0.009*"妹子" + 0.008*"爱" + 0.008*"心"

8 0.012*"美国" + 0.009*"娃" + 0.006*"想" + 0.006*"英国" + 0.005*"摄影师"

9 0.017*"吃惊" + 0.014*"梦" + 0.008*"视频" + 0.006*"女性" + 0.005*"面对"



In [95]:
key_words[0]

['市中心', '分', '泰国', '曼谷', '发生', '爆炸', '旅游景点', '记者', '现场', '伤亡', '推送']

In [96]:
'''
预测
'''
test_dict=word_dict.doc2bow(key_words[0])
res=lda[test_dict]
print(res)
np.sum([ttt[1] for ttt in res])

[(2, 0.6641561), (5, 0.15530746), (8, 0.119792685)]


0.93925625

In [97]:
'''
预测每个语料
'''
key_words_list=raw_data['key_words'].tolist()
cate_list=[]
for kw in key_words:
    test_dict=word_dict.doc2bow(kw)
    res=lda[test_dict]
    res.sort(key=lambda x:x[1],reverse=True)
    cate_list.append(res[0][0])
raw_data['topic_cate']=cate_list

In [99]:
raw_data['topic_cate'].value_counts()

1    1742
0    1316
6    1274
2    1152
3    1046
8    1024
5     985
7     940
4     930
9     926
Name: topic_cate, dtype: int64

In [100]:

raw_data[raw_data['topic_cate']==9]

,comment,like,repost,title,key_words,topic_cate
3,723,4774,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,"[妹纸, 图, 话, 说, 做, 素颜, 约会, 软件, 男人, 反差, 数量, 差, 感受]",9
48,522,2328,437,伦敦七夕怎么玩？ 敢不敢今晚带个妹纸去吃肉夹馍？？,"[伦敦, 玩, 妹纸, 吃, 肉夹馍]",9
49,79,889,231,【体验-伦敦】最近UCL建筑学院的学生把Kings Cross车站后面的荒地改造成了一个城市...,"[体验, 伦敦, 菜, 建筑, 学院, 学生, 车站, 荒地, 改造, 成, 城市园林, 邀...",9
117,77,659,247,【户外-伦敦】看着碟中谍5里阿汤哥酷炫的装备和超燃的任务场面，你是不是已经摩拳擦掌好久了？D...,"[特工, 伦敦, 看着, 碟中谍, 阿汤哥, 装备, 超燃, 场面, 摩拳擦掌, 梦, 学校...",9
125,369,2081,2470,油管上最近超级火的舞蹈，蛮酷的，律动感十足，搭上音乐感觉都要跳起来了，棒！！[赞]http/...,"[油管, 火, 舞蹈, 蛮酷, 律, 动感, 搭, 音乐, 感觉, 跳, 赞]",9
131,3016,3795,2002,今天莎莎在香港铜锣湾搞了个用支付宝就打五折活动，感觉一下子现场就沸腾了。。。。。。,"[莎莎, 香港, 铜锣湾, 搞了个, 支付宝, 感觉, 现场, 沸腾]",9
145,1243,4749,3380,一个法国艺术家Charles Petillion最近在伦敦Covent Garden建造的一...,"[伦敦, 气球, 心跳, 法国, 艺术家, 建造, 组织, 云, 装置, 名为, 心脏, 地...",9
153,1000,5279,921,因为在车里发现了一大包白色的物质... 一个哥们被警方以涉嫌持有冰毒逮捕被关了起来........,"[白色, 物质, 哥们, 警方, 发现, 大包, 涉嫌, 持有, 冰毒, 逮捕, 关, 放,...",9
164,43,249,450,1分钱抢个钻石王老五男友约会，还送1克拉钻石戒指！转发并@ 3位好友，即有机会获得@全城热恋...,"[钻石, 分钱, 抢, 王老五, 男友, 约会, 送, 钻石戒指, 转发, 位, 好友, 机会]",9
183,586,1538,1365,几个非常实用的神技能，分分钟学起来，拿去装逼吧！[doge],"[实用, 神, 技能, 分, 学, 装, 逼]",9


# 结果分析

    1、利用jieba提供的接口提取关键字，背后应该是采用TF-IDF进行提取的。我们人为的指定了词性。
    2、利用gensim提供的接口训练的LDA，并对每一个文本进行主题分类。
    3、发现其实主题分类的结果和人的直觉有一定差距。
    4、之后可能会希望从两个方面改进分类结果：
        4.1 提取关键词时，对词汇的词性再做更优化的限定。
        4.2 尝试使用较为精细的LDA分类方法，并使用阈值来过滤分类。
        4.3 是否需要考虑分类间的相似性？
        
    5、总的来看，LDA的效果并不是特别的理想，原因在哪里？
        5.1 细节上看，可能是我们对语料的清洗提取程度不够，无法给出合适的词汇。
        5.2 但是横向对比上，我觉得更有可能的是，语料本身并没有很直观的主题。
           LDA在新闻类的语料上效果可以，是因为新闻类语料的主题明显，每个主题下的词汇差异性比较大。
        5.3 考虑情感分析，对每条文本的情感情感进行分类。